In [1]:
import os

In [2]:
cwd = '/Users/mingyi_work/Desktop/Cognistx/amerigas/phase 3/DQE/05102021'
os.chdir(cwd)

# Get data

In [3]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt,ceil
from itertools import combinations
from sklearn.cluster import KMeans
import collections, numpy
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt
import seaborn as sns

## VWERK

In [4]:
df_knvv = pd.read_csv('KNVV-cleaned.TXT', sep='\t', usecols=["KNVV.KUNNR", "KNVV.VWERK"])
df_knvv.columns = [i.split('.')[1] for i in df_knvv.columns]

# filter ship to 
df_knvv.KUNNR = pd.to_numeric(df_knvv.KUNNR, errors='coerce')
df_knvv = df_knvv[~df_knvv.KUNNR.isna()]
df_knvv.KUNNR = df_knvv.KUNNR.astype('int64')

df_knvv = df_knvv[df_knvv.VWERK.isin([9225, 3221, 5171, 5185, 5228, 5169, 5511])]

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Geocode

'GeoCodeDistribution2Year_2020-09-xx'  is the 2 year extract  
"GeoCodeExtract_2020-09-14" is daily extract of last 15 days. we get this everyday  

Fields :

- Ship to  
- Delivery date / Route date
- Order number  
- Order Date  
- Gallons/Cylinders delivered  

If no delivery:  

- Report gallons/cylinders are 0  
- Delivery location ping geocode  

### 15days

In [5]:
# 2021-02-11 - 2021-02-25
ext1=pd.read_csv('/Users/mingyi_work/Desktop/Cognistx/amerigas/data/geocode/GeoCodeExtract_02262021.TXT',sep='\t')

ext1=ext1[(ext1.Cylinders_Delivered!=0)&(~ext1.Cylinders_Delivered.isna())]
ext1.Route_Date=pd.to_datetime(ext1.Route_Date)
ext1=ext1.sort_values('Route_Date')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# 2021-02-26 - 2021-03-12
ext2=pd.read_csv('/Users/mingyi_work/Desktop/Cognistx/amerigas/data/geocode/GeoCodeExtract_03132021.TXT',sep='\t')

ext2=ext2[(ext2.Cylinders_Delivered!=0)&(~ext2.Cylinders_Delivered.isna())]
ext2.Route_Date=pd.to_datetime(ext2.Route_Date)
ext2=ext2.sort_values('Route_Date')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
ext3=pd.read_csv('/Users/mingyi_work/Desktop/Cognistx/amerigas/data/geocode/GeoCodeExtract_03252021.TXT',sep='\t')

ext3=ext3[(ext3.Cylinders_Delivered!=0)&(~ext3.Cylinders_Delivered.isna())]
ext3.Route_Date=pd.to_datetime(ext3.Route_Date)
ext3=ext3.sort_values('Route_Date')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
ext4=pd.read_csv('/Users/mingyi_work/Desktop/Cognistx/amerigas/data/geocode/GeoCodeExtract_04062021.TXT',sep='\t')

ext4=ext4[(ext4.Cylinders_Delivered!=0)&(~ext4.Cylinders_Delivered.isna())]
ext4.Route_Date=pd.to_datetime(ext4.Route_Date)
ext4=ext4.sort_values('Route_Date')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
ext5=pd.read_csv('/Users/mingyi_work/Desktop/Cognistx/amerigas/data/geocode/GeoCodeExtract_04182021.TXT',sep='\t')

ext5=ext5[(ext5.Cylinders_Delivered!=0)&(~ext5.Cylinders_Delivered.isna())]
ext5.Route_Date=pd.to_datetime(ext5.Route_Date)
ext5=ext5.sort_values('Route_Date')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
ext6=pd.read_csv('/Users/mingyi_work/Desktop/Cognistx/amerigas/data/geocode/GeoCodeExtract_05022021.TXT',sep='\t')

ext6=ext6[(ext6.Cylinders_Delivered!=0)&(~ext6.Cylinders_Delivered.isna())]
ext6.Route_Date=pd.to_datetime(ext6.Route_Date)
ext6=ext6.sort_values('Route_Date')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
ext0=pd.read_csv('GeoCodeExtract.TXT',sep='\t')

ext0=ext0[(ext0.Cylinders_Delivered!=0)&(~ext0.Cylinders_Delivered.isna())]
ext0.Route_Date=pd.to_datetime(ext0.Route_Date)
ext0=ext0.sort_values('Route_Date')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
ext = pd.concat([ext0,ext1, ext2, ext3, ext4,ext5,ext6], axis = 0)
ext = ext.drop_duplicates()

# filter ship to
ext = ext[ext.Customer_Ship_To_Number < 2 *10**8]

#filter sold to
ext = ext[(ext.Cylinders_Delivered != 0)]
ext = ext[(ext.Actual_Gallons > 0) & (~ext.Actual_Gallons.isna())]
ext = ext[~ext.Cluster_Latitude.isna()]

# route_Date datatype change
ext.Route_Date=pd.to_datetime(ext.Route_Date)
ext=ext.sort_values('Route_Date')

### 2 years data
* date: 2017-10-01 --- 2021-02-11

* KUNNR: 1. only include integer(e.g AGOWN is deleted); 2. smaller than 200,000,000

* Actual_Gallons > 0 and is not missing
* Cylinders_Delivered != 0
* cluster_latitude is not missing

In [13]:
dis = pd.read_csv('/Users/mingyi_work/Desktop/Cognistx/amerigas/data/geocode/GeoCodeExtract_Full.csv')
dis = dis.drop(columns = ['Unnamed: 0'])

# filter ship to
dis['Customer_Ship_To_Number'] = pd.to_numeric(dis['Customer_Ship_To_Number'], errors='coerce').astype('int64')
dis = dis[~dis.Customer_Ship_To_Number.isna()]
dis = dis[dis.Customer_Ship_To_Number<2*10**8]

#filter sold to
dis = dis[(dis.Cylinders_Delivered != 0)]
dis = dis[(dis.Actual_Gallons > 0) & (~dis.Actual_Gallons.isna())]
dis = dis[~dis.Cluster_Latitude.isna()]

# route_Date datatype change
dis.Route_Date=pd.to_datetime(dis.Route_Date)
dis=dis.sort_values('Route_Date')

dis = dis.reset_index(drop = True)

In [14]:
print('Two years data:',dis.Route_Date.min(),'-',dis.Route_Date.max())

Two years data: 2017-10-01 00:00:00 - 2021-02-11 00:00:00


In [15]:
print('dis', dis.shape)
print(dis.isnull().sum(0)/dis.shape[0])

dis (18360488, 9)
Route_Date                 0.000000
Customer_Ship_To_Number    0.000000
Sales_Order_Number         0.000000
Created_On_Order           0.000000
Cluster_Latitude           0.000000
Cluster_Longitude          0.000000
Actual_Gallons             0.000000
Cylinders_Delivered        0.750352
DC_Order_Reason            0.806141
dtype: float64


### Combine geocode data

In [16]:
df_geocode = pd.concat([ext, dis[ext.columns]], axis = 0)
df_geocode = df_geocode.drop_duplicates()

In [17]:
# Only keep accounts in valid district
df_geocode = df_geocode[df_geocode.Customer_Ship_To_Number.isin(df_knvv.KUNNR)]

# Exclude YARD 

In [18]:
class AppConstant(object):
    MELISSA_API_BATCH_LIMIT = 100
    MELISSA_DEFAULT_COUNTRY = "USA"
    DEFAULT_TIMEZONE = "US/Eastern"
    RESOURCE_S3 = "s3"
    USECASE_PROCESSING_BATCH_LIMIT = 10000
    LOAD_CUSTOMERS_BATCH_LIMIT = 50000
    S436_BATCH_LIMIT = 100000
    ANOMALY_STATUS_REPEAT = 'REPEAT'
    ANOMALY_STATUS_RESOLVED = 'RESOLVED'
    RULE_TYPE_INDIVIDUAL = 'INDIVIDUAL'
    RULE_TYPE_COMBINED = 'COMBINED'
    SIMILAR_GEOCODES = 3
    DUPLICATE_GEOCODES = 2
    GEOCODES_LENGTH = 250 # In feets
    YARD_LENGTH = 250 # In feets
    RADIUS_OF_EARTH = 20914080 # In Feets
    RULE_SIMILAR_GEOCODE_MATCHED = -2
    RULE_SIMILAR_GEOCODE_LENGTH_3 = 1
    RULE_SIMILAR_GEOCODE_LENGTH_2 = 2
    RULE_SIMILAR_GEOCODE_LENGTH_0 = 4
    GEOCODE_PROCESSING_BATCH_LIMIT = 50000

In [19]:
def get_distance(lon1, lat1, lon2, lat2):
        """
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees)
        """
        # convert decimal degrees to radians
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        # haversine formula
        dist_long = lon2 - lon1
        dist_lat = lat2 - lat1
        a = sin(dist_lat/2)**2 + cos(lat1) * cos(lat2) * sin(dist_long/2)**2
        c = 2 * asin(sqrt(a))
        # Radius of earth in feets and hence distance in feets
        dist_ft = AppConstant.RADIUS_OF_EARTH * c
        return dist_ft

In [20]:
def get_distance_vector(df_yard,lon1, lat1):
    yard_long = np.array(list(map(radians, df_yard.Longitude)))
    yard_lat = np.array(list(map(radians, df_yard.Latitude)))
    lon1 = list(map(radians, [lon1]))
    lat1 = list(map(radians, [lat1]))
    mylon = yard_long - lon1
    mylat = yard_lat - lat1
    a = np.sin(mylat/2)**2 + np.cos(yard_lat) * np.cos(lat1) * np.sin(mylon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    dist_ft = AppConstant.RADIUS_OF_EARTH * c
    return min(dist_ft)

In [21]:
df_yard = pd.read_excel('/Users/mingyi_work/Desktop/Cognistx/amerigas/AmeriGas AI:ML/geocode cluster/Fill Location Geocodes 04222021.xlsx', sheet_name='Master List')
df_yard.columns = [i.strip() for i in df_yard.columns]

In [22]:
df_yard.Longitude = df_yard.Longitude.astype(str).str.strip().replace('',np.nan)
df_yard.Latitude = df_yard.Latitude.astype(str).str.strip().replace('',np.nan)
df_yard = df_yard[~(df_yard.Latitude.isna() | df_yard.Longitude.isna())]

df_yard.Longitude = df_yard.Longitude.astype('float64')
df_yard.Latitude = df_yard.Latitude.astype('float64')

df_yard = df_yard[['Longitude','Latitude']].drop_duplicates()

In [23]:
df_geocode = df_geocode.reset_index(drop=True)
df_yard = df_yard.reset_index(drop=True)
df_geocode['yard_flag'] = 0
for n in range(df_geocode.shape[0]):
    if n % 10000 == 0:
        print (n)
    tmp_distance = get_distance_vector(df_yard,df_geocode.Cluster_Longitude.iloc[n], df_geocode.Cluster_Latitude.iloc[n])
    if tmp_distance <= AppConstant.YARD_LENGTH:
        df_geocode['yard_flag'].iloc[n] = 1

0


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000


# format to fit following function

In [24]:
dis10=df_geocode[df_geocode.yard_flag == 0].sort_values(['Customer_Ship_To_Number','Route_Date','Sales_Order_Number'],ascending = False).groupby('Customer_Ship_To_Number').head(10)

In [25]:
dis10.groupby('Customer_Ship_To_Number').Route_Date.count().value_counts()

10    3858
1     3306
2     2303
3     1912
4     1220
5     1004
6      787
7      640
8      549
9      436
Name: Route_Date, dtype: int64

In [26]:
dis10['geo_combine'] = dis10[['Cluster_Longitude','Cluster_Latitude']].values.tolist()

In [27]:
dis_geo_combine = dis10.groupby('Customer_Ship_To_Number')['geo_combine'].apply(list).reset_index()
dis_geo_combine['geo_count'] = dis_geo_combine['geo_combine'].apply(lambda x: len(x))

# get 3 closest geocodes based on 10 latest deliveres
* distance  = 250

In [28]:
def get_similar_geocodes(geocodes,geo_length):
    #logger.debug("Finding geocodes which are similar or whose length is less than %s", AppConstant.GEOCODES_LENGTH)
    duplicate = None
  
    # Checking if geocodes are similar
    if (geocodes is not None and len(geocodes) >= AppConstant.SIMILAR_GEOCODES):
        #logger.debug("Got geocodes with length  %s", len(geocodes))
    
        # List for Similar geocodes
        geocode_list = list()
        for item in geocodes:
            if geocodes.count(item) >= AppConstant.SIMILAR_GEOCODES:
                #logger.debug("Got similar geocodes with value %s", item)
                geocode_list += AppConstant.SIMILAR_GEOCODES * [item]
                return geocode_list
            if duplicate is None and geocodes.count(item) == AppConstant.DUPLICATE_GEOCODES:
                #logger.debug("Got duplicate geocodes with value %s", item)
                # List for duplicate geocodes
                geo_duplicate_list = list()
                geo_duplicate_list += AppConstant.DUPLICATE_GEOCODES * [item]
                duplicate = geo_duplicate_list
    else:
        #logger.debug("Nothing to process since got geocodes data with length less than %s.", AppConstant.SIMILAR_GEOCODES)
        return list()

    min_len_pair_val = float('inf') 
    min_len_pair_geocodes = None

    # Finding unique combinations of geocodes by length specified(AppConstant.SIMILAR_GEOCODES) 
    geocode_combinations_tup = combinations(geocodes, AppConstant.SIMILAR_GEOCODES)
    geocode_combinations = list(geocode_combinations_tup)
    
    if len(geocode_combinations)> 0:
        geocode_comb_list = list(geocode_combinations)
        no_of_elements = 2
        
        # Iterating over unique geocode combination list and finding a combination
        for geocode_comb in geocode_comb_list:
            # Getting combinations of inner elements and converting to list
            comb = combinations(list(geocode_comb), no_of_elements)
            comb_list = list(comb)
            
            if comb_list is not None and len(comb_list)> 0:
                for index, item in enumerate(comb_list):
                    comb_item = list(item)
                    distance = get_distance(comb_item[0][0], comb_item[0][1], comb_item[1][0], comb_item[1][1])
                    
                    if distance > geo_length:
                        break
    
                    # Finding two geocodes satisfying the criteria
                    if distance < min_len_pair_val:
                        min_len_pair_val = distance
                        min_len_pair_geocodes = item
                    
                    if index == len(comb_list)-1:
                        #logger.debug("Got geocodes satisfying the criteria of having length less than %s", AppConstant.GEOCODES_LENGTH)
                        return geocode_comb
        
        #logger.debug("No %s geocodes exist which satisfied the criteria", AppConstant.SIMILAR_GEOCODES)
    
        # Check if geocodes with similar value already exists
        if duplicate is not None:
            #logger.debug("Got similar geocodes with duplicate values and hence returning")
            return duplicate
    
        # If there are atleast two points whose distance between them satisfied the criteria
        if min_len_pair_geocodes is not None:
            #logger.debug("Got two geocodes whose length is minimum and satisfied the criteria")
            return min_len_pair_geocodes
    
        # Checking by making combinations of two in the original array
        geocodes_comb_pair_tup = combinations(geocodes, no_of_elements)
        geocodes_comb_pair = list(geocodes_comb_pair_tup)
       
        #self.logger.debug("Got geocode combination pairs with length %d", len(geocodes_comb_pair), exc_info=1)
        for geocode_comb in geocodes_comb_pair:
            item = list(geocode_comb)
            distance = get_distance(item[0][0], item[0][1], item[1][0], item[1][1])
    
            if distance < geo_length:
                #logger.debug("Got geocode pairs whose distance between them satisfied the criteria")
                return geocode_comb
                
    else:
        #logger.debug("No geocode combinations found")
        return list()              
        
        
            
        


### Generate 3 geocodes

In [29]:
dis_geo_combine['geo_3_close'] = dis_geo_combine['geo_combine'].apply(lambda x: get_similar_geocodes(x,250))

dis_geo_combine = dis_geo_combine[~dis_geo_combine.geo_3_close.isna()]
dis_geo_combine = dis_geo_combine[dis_geo_combine.geo_3_close.str.len()!=0]

In [30]:
def avg(list):
    return sum(list)/len(list)

In [31]:
dis_geo_combine['recommend'] = dis_geo_combine.geo_3_close.apply(lambda x: list(map(avg, zip(*x))))
dis_geo_combine['recommend'] = dis_geo_combine.recommend.apply(lambda x: [round(i,10) for i in x])

In [32]:
dis_geo_combine['long_recommend'] = dis_geo_combine['recommend'].apply(lambda x: str(x[0]))
dis_geo_combine['lat_recommend'] = dis_geo_combine['recommend'].apply(lambda x: str(x[1]))

## Compare with SAP lat&long

In [33]:
df_latlong = pd.read_csv('KNA1-cleaned.TXT', sep = '\t')[['KNA1./ICO/KOORDX','KNA1./ICO/KOORDY','KNA1.KUNNR']]
df_latlong.columns = ['SAP_long','SAP_lat','Customer_Ship_To_Number']
df_latlong = df_latlong[(~df_latlong['SAP_lat'].isnull()) & (~df_latlong['SAP_long'].isnull())]

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
out = dis_geo_combine.merge(df_latlong, on = 'Customer_Ship_To_Number', how = 'left')
out = out.merge(df_knvv.rename(columns = {'KUNNR':'Customer_Ship_To_Number'}), on = 'Customer_Ship_To_Number', how = 'left')

In [35]:
out['away_from_SAP'] = out.apply(lambda x: get_distance(x.SAP_long, x.SAP_lat, float(x.long_recommend), float(x.lat_recommend)), axis = 1)

In [36]:
out_pilot_3 = out[['Customer_Ship_To_Number','VWERK','geo_3_close','recommend','long_recommend','lat_recommend','SAP_long','SAP_lat','away_from_SAP']]
out_pilot_3.columns = ['Customer Ship To Number','District','Top 3 Closest Geocodes','Geocode Recommendation','Longitude recommendation','Latitude Recommendation','SAP longitude','SAP latitude','Distance from SAP']

In [ ]:
out_pilot_3.to_csv('/Users/mingyi_work/Desktop/Cognistx/amerigas/data/geocode/geocode_recommmendation_pilot4.csv')

In [ ]:
# delete geo_3_close == []
df_sap = df_sap[df_sap['geo_3_close'].str.len() != 0]

In [ ]:
df_sap[['geo1','geo2','geo3']] = pd.DataFrame(df_sap['geo_3_close'].tolist(),index = df_sap.index)

In [ ]:
df_sap['geo_count'].value_counts()

In [ ]:
df_sap['len1'] = df_sap.apply(lambda x: get_distance(x.SAP_long, x.SAP_lat, x.geo1[0], x.geo1[1]) if (x.geo1 is not None) else None, axis = 1)
df_sap['len2'] = df_sap.apply(lambda x: get_distance(x.SAP_long, x.SAP_lat, x.geo2[0], x.geo2[1]) if (x.geo2 is not None) else None, axis = 1)
df_sap['len3'] = df_sap.apply(lambda x: get_distance(x.SAP_long, x.SAP_lat, x.geo3[0], x.geo3[1]) if (x.geo3 is not None) else None, axis = 1)


In [ ]:
df_sap['len_mean'] = df_sap[['len1','len2','len3']].mean(axis = 1)

In [ ]:
df_sap.head()

In [ ]:
df_sap.len_mean.describe()

#### 400 feet distance tolerance

In [ ]:
df_sap_400 = dis_geo_combine_400.merge(df_latlong, on = 'Customer_Ship_To_Number', how = 'left')
df_sap_400 = df_sap_400[~df_sap_400['SAP_long'].isnull()]
df_sap_400 = df_sap_400[~df_sap_400['geo_3_close'].isnull()]
df_sap_400 = df_sap_400[df_sap_400['geo_3_close'].str.len() != 0]
df_sap_400[['geo1','geo2','geo3']] = pd.DataFrame(df_sap_400['geo_3_close'].tolist(),index = df_sap_400.index)

df_sap_400['len1'] = df_sap_400.apply(lambda x: get_distance(x.SAP_long, x.SAP_lat, x.geo1[0], x.geo1[1]) if (x.geo1 is not None) else None, axis = 1)
df_sap_400['len2'] = df_sap_400.apply(lambda x: get_distance(x.SAP_long, x.SAP_lat, x.geo2[0], x.geo2[1]) if (x.geo2 is not None) else None, axis = 1)
df_sap_400['len3'] = df_sap_400.apply(lambda x: get_distance(x.SAP_long, x.SAP_lat, x.geo3[0], x.geo3[1]) if (x.geo3 is not None) else None, axis = 1)
df_sap_400['len_mean'] = df_sap_400[['len1','len2','len3']].mean(axis = 1)

In [ ]:
df_sap_400.len_mean.describe()

In [ ]:
df_sap_400_no_anomaly = df_sap_400[df_sap_400.len_mean < 10000]

## Delete anomaly before clustering 

In [ ]:
# % of range
df_sap_dist = np.log10(df_sap['len_mean']).apply(np.floor).value_counts().reset_index()
df_sap_dist.columns = ['range','count']
df_sap_dist['percentage%'] = df_sap_dist['count']*100 / df_sap.shape[0]
df_sap_dist = df_sap_dist.sort_values('range')
df_sap_dist.insert(1,column = 'value', value = ['0-1','1-10','10-100','100-1000','1000-10000','10000-1e5','1e5-1e6','1e6-1e7','1e7-1e8'])
print(df_sap_dist)

In [ ]:
# delete len_mean > 10000 
# they take 2.32% of all data
df_sap_no_anomaly = df_sap[df_sap.len_mean<10000]

## K-means clustering

#### 250 feet tolerance

In [ ]:
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(df_sap_no_anomaly['len_mean'].values.reshape(-1,1))
    kmeanModel.fit(df_sap_no_anomaly['len_mean'].values.reshape(-1,1))
    distortions.append(sum(np.min(cdist(df_sap_no_anomaly['len_mean'].values.reshape(-1,1), kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / df_sap_no_anomaly.shape[0])

In [ ]:
# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

#### 400 feet tolerance

In [ ]:
distortions_400 = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(df_sap_400_no_anomaly['len_mean'].values.reshape(-1,1))
    kmeanModel.fit(df_sap_400_no_anomaly['len_mean'].values.reshape(-1,1))
    distortions_400.append(sum(np.min(cdist(df_sap_400_no_anomaly['len_mean'].values.reshape(-1,1), kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / df_sap_no_anomaly.shape[0])

In [ ]:
# Plot the elbow
plt.plot(K, distortions_400, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

## Decide # of groups

In [ ]:
kmeans = KMeans(n_clusters=5, max_iter=300, n_init=10, random_state=0)
df_sap_no_anomaly['group'] = kmeans.fit_predict(df_sap_no_anomaly['len_mean'].values.reshape(-1,1))

In [ ]:
df_sap_no_anomaly['group'] = df_sap_no_anomaly['group'].replace({0:1,1:3,2:0,3:2,4:4})

In [ ]:
out = df_sap_no_anomaly.groupby('group').head(500)
out = out[['Customer_Ship_To_Number','SAP_long','SAP_lat','geo1','geo2','geo3','group']]

In [ ]:
out.to_csv('/Users/mingyi_work/Desktop/Cognistx/amerigas/AmeriGas AI:ML/geocode cluster/geocode_cluster_500.csv')

In [ ]:
df_sap_no_anomaly.groupby('group')['group'].count()

In [ ]:
df_group = df_sap_no_anomaly.groupby('group').agg({'len_mean':['min','max','mean','count']})
df_group.columns = ['min','max','mean','count']
df_group = df_group.sort_values('min').reset_index(drop = True)

#### 5 Groups

In [ ]:
df_group.head()

In [ ]:
#kmeans.cluster_centers_

#### 4 groups

In [ ]:
kmeans = KMeans(n_clusters=4, max_iter=300, n_init=10, random_state=0)
df_sap_no_anomaly['group'] = kmeans.fit_predict(df_sap_no_anomaly['len_mean'].values.reshape(-1,1))
df_group = df_sap_no_anomaly.groupby('group').agg({'len_mean':['min','max','mean','count']})
df_group.columns = ['min','max','mean','count']
df_group = df_group.sort_values('min').reset_index(drop = True)
df_group

#### 3 groups

In [ ]:
kmeans = KMeans(n_clusters=3, max_iter=300, n_init=10, random_state=0)
df_sap_no_anomaly['group'] = kmeans.fit_predict(df_sap_no_anomaly['len_mean'].values.reshape(-1,1))
df_group = df_sap_no_anomaly.groupby('group').agg({'len_mean':['min','max','mean','count']})
df_group.columns = ['min','max','mean','count']
df_group = df_group.sort_values('min').reset_index(drop = True)
df_group

### 400 feet tolerance

In [ ]:
kmeans = KMeans(n_clusters=5, max_iter=300, n_init=10, random_state=0)
df_sap_400_no_anomaly['group'] = kmeans.fit_predict(df_sap_400_no_anomaly['len_mean'].values.reshape(-1,1))
df_group_400 = df_sap_400_no_anomaly.groupby('group').agg({'len_mean':['min','max','mean','count']})
df_group_400.columns = ['min','max','mean','count']
df_group_400 = df_group_400.sort_values('min').reset_index(drop = True)
df_group_400.head()